In [1]:
!pip install accelerate
!pip install langchain-community
!pip install langchain
!pip install sentence-transformers
!pip install pdfplumber
!pip install chromadb
!pip install tiktoken

In [2]:
!pip install --upgrade langchain

In [3]:
!pip install docx2txt

In [4]:
import torch
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter
from langchain.vectorstores import Chroma
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders.word_document import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

2025-07-05 09:04:14.461042: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751706254.483749     451 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751706254.490726     451 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
embedding_model = HuggingFaceEmbeddings(
    model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

/tmp/ipykernel_451/3529680270.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [6]:
file_path = "/kaggle/input/dataforllm2/data4.docx"

In [7]:
try:
    loader = Docx2txtLoader(file_path)
    documents = loader.load()
    print(f"Loaded {len(documents)} document(s).")
    if documents:
        print(documents[0].page_content[2000:4000])
except Exception as e:
    print(f"Error loading the file: {e}")

Loaded 1 document(s).
 قضای آن واجب نیست.

یکی از شرایط نماز پاک بودن اعضای بدن ازجمله مو و ناخن و همچنین لباس نمازگزار است، ولی لباس‌های کوچک مانند جوراب و کلاه که به‌تنهایی نمی‌توانند عورت انسان را بپوشانند لازم نیست پاک باشند، البته به‌شرط این‌که ـ بنابر احتیاط واجب ـ از مردارِ نجس یا حیوان نجس العین مانند سگ درست نشده باشند، و همراه داشتن شیء متنجس در حال نماز مثل قرار دادن آن در جیب اشکال ندارد.

اگر در بدن یا لباس، خونِ زخم یا دمل باشد تا زمانی که التیام نیافته باشد نمازخواندن با آن مانعی ندارد ولی زخم باید قابل‌توجه باشد، و اما زخم‌های سطحی را باید تطهیر کرد.

اگر بدن یا لباس کسی آلوده به خون شود درحالی‌که مقدار آن کمتر از بند سرانگشت شست باشد نمازخواندن با آن صحیح است، البته این حکم در مواردی استثنا شده ازجمله: خون حیض و همچنین بنابر احتیاط واجب خون نجس العین و خون مردار و خون حیوان حرام‌گوشت و خون نفاس و خون استحاضه.

اگر شخصی از نجاست بدن یا لباس خود بی‌اطلاع بود و با آن نماز خواند و پس از تمام شدن نماز از نجاستش مطلع شود نماز او صحیح است، ولی اگر قبل از نماز به نجاست آن شک د

In [8]:
try:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=700,
        chunk_overlap=100,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Created {len(chunks)} chunks.")
    if chunks:
        print(chunks[0])
except Exception as e:
    print(f"Error chunking the document: {e}")

Created 10 chunks.
page_content='مهم‌ترین نمازهای واجب نمازهای روزانه پنج‌گانه می‌باشد

نماز صبح: دو رکعت

نماز ظهر: چهار رکعت

نماز عصر: چهار رکعت

نماز مغرب: سه رکعت

نماز عشا: چهار رکعت

و در شرایطی معین نمازهای چهار رکعتی در سفر یا هنگام ترس، شکسته خوانده می‌شوند.

مستحب است در هر شبانه‌روز سی‌وچهار رکعت نماز نافله به‌جا آورده شود که عبارتند از

هشت رکعت قبل از نماز ظهر

هشت رکعت قبل از نماز عصر

چهار رکعت بعد از نماز مغرب

دو رکعت بعد از نماز عشا که بنابر احتیاط واجب، باید نشسته خوانده شود و یک رکعت حساب می‌شود

هشت رکعت نماز شب، و وقت آن از اول شب آغاز می‌شود، به این معنا که اگر بعد از نماز عشا آن را بخواند، در وقت واقع شده است و بهتر است نزدیک طلوع فجر خوانده شود

دو رکعت نماز شفع بعد از نماز شب' metadata={'source': '/kaggle/input/dataforllm2/data4.docx'}


In [9]:
db = Chroma.from_documents(chunks, embedding_model, persist_directory = "/kaggle/working/")

In [10]:
retriever = db.as_retriever(search_kwargs = {"k": 1})

In [11]:
llm_name = "MehdiHosseiniMoghadam/AVA-Llama-3-V2"

In [12]:
tokenizer = AutoTokenizer.from_pretrained(llm_name)

In [13]:
model = AutoModelForCausalLM.from_pretrained(
    llm_name, 
    device_map = "auto",
    torch_dtype = torch.float16
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [14]:
llm_pipeline = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    max_new_tokens = 256,
    do_sample = True,
    temperature = 0.7,
    top_p = 0.9
)

Device set to use cuda:0


In [15]:
llm = HuggingFacePipeline(pipeline = llm_pipeline)

/tmp/ipykernel_451/1706921424.py:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline = llm_pipeline)


In [16]:
prompt_template = PromptTemplate(
    input_variables = ["context", "question"], template = """
    با توجه به متن زیر به پرسش پاسخ بده:
    متن:
    {context}

    پرسش:
    {question}

    پاسخ را به زبان فارسی روان بده:
    """
)

In [17]:
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = retriever,
    chain_type_kwargs = {"prompt":
                        prompt_template}
)

In [19]:
question = "وقت نماز صبح کی است؟"
result = qa_chain.run(question)
print("پرسش:", question)
print("پاسخ:", result)

پرسش: وقت نماز صبح کی است؟
پاسخ: 
    با توجه به متن زیر به پرسش پاسخ بده:
    متن:
    دو رکعت نماز شفع بعد از نماز شب

یک رکعت نماز وتر بعد از نماز شفع

دو رکعت قبل از نماز صبح

به غیر از نماز وتر که یک‌رکعتی است بقیه نمازهای نافله دو رکعت دو رکعت خوانده می‌شود

وقت نماز صبح از طلوع فجر صادق تا طلوع آفتاب، و وقت نماز ظهر و عصر از زوال آفتاب تا غروب آن است، و نماز ظهر باید قبل از نماز عصر خوانده شود و زوال، نصف فاصله زمانی بین طلوع آفتاب تا غروب آن می‌باشد، و وقت نماز مغرب و عشا برای شخص مختار از اول مغرب تا نیمه‌شب است و نیمه‌شب، نصف فاصله زمانی بین غروب آفتاب تا طلوع فجر می‌باشد، و اگر مکلف به اختیار خود نماز مغرب و عشا را تا نیمه‌شب نخواند بنابر احتیاط واجب باید قبل از طلوع فجر بدون نیت ادا یا قضا آن را به‌جا آورد، و در هر دو حالت باید نماز مغرب قبل از عشا خوانده شود.

    پرسش:
    وقت نماز صبح کی است؟

    پاسخ را به زبان فارسی روان بده:
     وقت نماز صبح از طلوع فجر صادق تا طلوع آفتاب است.


In [20]:
question = "اگر بدن یا لباس کسی آلوده به خون شود درحالی‌که مقدار آن کمتر از بند سرانگشت شست باشد"
result = qa_chain.run(question)
print("پرسش:", question)
print("پاسخ:", result)

پرسش: اگر بدن یا لباس کسی آلوده به خون شود درحالی‌که مقدار آن کمتر از بند سرانگشت شست باشد
پاسخ: 
    با توجه به متن زیر به پرسش پاسخ بده:
    متن:
    اگر در بدن یا لباس، خونِ زخم یا دمل باشد تا زمانی که التیام نیافته باشد نمازخواندن با آن مانعی ندارد ولی زخم باید قابل‌توجه باشد، و اما زخم‌های سطحی را باید تطهیر کرد.

اگر بدن یا لباس کسی آلوده به خون شود درحالی‌که مقدار آن کمتر از بند سرانگشت شست باشد نمازخواندن با آن صحیح است، البته این حکم در مواردی استثنا شده ازجمله: خون حیض و همچنین بنابر احتیاط واجب خون نجس العین و خون مردار و خون حیوان حرام‌گوشت و خون نفاس و خون استحاضه.

اگر شخصی از نجاست بدن یا لباس خود بی‌اطلاع بود و با آن نماز خواند و پس از تمام شدن نماز از نجاستش مطلع شود نماز او صحیح است، ولی اگر قبل از نماز به نجاست آن شک داشت و بدون آن‌که بررسی کند با همان حال نماز خواند و بعد از نماز متوجه نجاست شود بنابر احتیاط واجب باید نماز را دوباره بخواند.

    پرسش:
    اگر بدن یا لباس کسی آلوده به خون شود درحالی‌که مقدار آن کمتر از بند سرانگشت شست باشد

    پاسخ را به زبان فارسی 

In [22]:
question = "پوشیدن لباس طلا و ابریشم خالص برای مرد چه حکمی دارد؟"
result = qa_chain.run(question)
print("پرسش:", question)
print("پاسخ:", result)

پرسش: پوشیدن لباس طلا و ابریشم خالص برای مرد چه حکمی دارد؟
پاسخ: 
    با توجه به متن زیر به پرسش پاسخ بده:
    متن:
    لباس مرد نمازگزار که برای پوشاندن عورت نمازگزار کافی است، باید از ابریشم طبیعی خالص نباشد.

لباس مرد نمازگزار از طلای خالص یا ناخالص نباشد، و همچنین پوشیدن طلا و زیورآلات مانند زنجیر و انگشترِ طلا نماز او را باطل می‌کند.

پوشیدن لباس طلا و ابریشم خالص برای مرد همیشه حرام است حتی در غیر نماز، بلکه بنابر احتیاط واجب مرد هیچ‌گاه نباید از طلا برای زینت خود استفاده کند.

    پرسش:
    پوشیدن لباس طلا و ابریشم خالص برای مرد چه حکمی دارد؟

    پاسخ را به زبان فارسی روان بده:
     پوشیدن لباس طلا و ابریشم خالص برای مرد در نماز حرام است و حتی در غیر نماز نیز بنابر احتیاط واجب مرد نباید از طلا برای زینت خود استفاده کند.


In [23]:
question = "اگر کسی در حال خواندن نماز متوجه شود عورتش پوشیده نیست باید چه کند؟"
result = qa_chain.run(question)
print("پرسش:", question)
print("پاسخ:", result)

پرسش: اگر کسی در حال خواندن نماز متوجه شود عورتش پوشیده نیست باید چه کند؟
پاسخ: 
    با توجه به متن زیر به پرسش پاسخ بده:
    متن:
    صورت به مقداری که روسری معمولاً آن را نمی‌پوشاند بااینکه روسری را گریبان زده باشد.

دست‌ها از مچ دست تا سر انگشتان.

پاها از مچ پا به پایین (در صورت عدم وجود نامحرم).

اگر کسی در حال خواندن نماز متوجه شود عورتش پوشیده نیست باید فوراً آن را بپوشاند و نمازش صحیح است، و اگر بعد از تمام کردنِ نماز متوجه این موضوع شود بازهم نماز او صحیح است.

شرایط لباس نمازگزار عبارتند از

پاک باشد، همان‌طور که گذشت.

لباسی که عورت را پوشانده بنابر احتیاط واجب مباح باشد.

لباسی که برای پوشاندن عورت به‌تنهایی کافی است نباید از اجزای دارای روح مردار باشد مانند پوست حیوانی که به روش غیرشرعی ذبح شده است، و همچنین بنابر احتیاط واجب اگر این لباس به‌تنهایی برای پوشاندن عورت کافی نباشد.

از اجزای درندگان ـ مانند روباه ـ نباشد، بلکه بنابر احتیاط واجب از اجزای حیوان حرام‌گوشت ـ مانند خرگوش و گربه ـ نباشد، و این شرط به لباسی اختصاص دارد که برای پوشاندن عورت نمازگزار کافی باشد و شامل غ

In [26]:
question = "اگر مرد و زن بخواهند در یک مکان نماز بخوانند چه کار باید کنند؟"
result = qa_chain.run(question)
print("پرسش:", question)
print("پاسخ:", result)

پرسش: اگر مرد و زن بخواهند در یک مکان نماز بخوانند چه کار باید کنند؟
پاسخ: 
    با توجه به متن زیر به پرسش پاسخ بده:
    متن:
    اگر مرد و زن بخواهند در یک مکان نماز بخوانند، بنابر احتیاط واجب زن نباید جلوتر از مرد یا مساوی با او بایستد، بلکه لازم است زن عقب‌تر از مرد بایستد و این عقب ایستادن حداقل به مقداری باشد که جای سجده او برابر جای دو زانوی مرد در حال سجده باشد، یا بین آن‌ها حائلی مانند دیوار وجود داشته باشد، یا این‌که فاصله بین آن دو بیش از ده ذراع (تقریباً چهار متر ونیم) باشد.

مکلف باید در حال نماز عورت خود را بپوشاند، و عورت مرد در نماز آلت تناسلی و بیضه‌ها و مقعد اوست، و عورت زن در نماز تمام بدن او به‌جز سه مورد می‌باشد:

صورت به مقداری که روسری معمولاً آن را نمی‌پوشاند بااینکه روسری را گریبان زده باشد.

دست‌ها از مچ دست تا سر انگشتان.

پاها از مچ پا به پایین (در صورت عدم وجود نامحرم).

    پرسش:
    اگر مرد و زن بخواهند در یک مکان نماز بخوانند چه کار باید کنند؟

    پاسخ را به زبان فارسی روان بده:
     در این مورد، بنابر احتیاط واجب زن نباید جلوتر از مرد یا مساوی با او بای